In [36]:
%pip install yfinance plotly --quiet

Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import scipy.optimize as sco
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

# Output dir
OUTDIR = "portfolio_output"
os.makedirs(OUTDIR, exist_ok=True)

# Trading days (for annualization)
TRADING_DAYS = 252

# Risk-free (annual) used for Sharpe; change if you want
RISK_FREE = 0.03


In [38]:
# Top 2 stocks from 5 sectors:
# IT: INFY.NS, TCS.NS
# Bank: HDFCBANK.NS, ICICIBANK.NS
# FMCG: ITC.NS, HINDUNILVR.NS
# Oil & Gas: RELIANCE.NS, ONGC.NS
# Pharma: SUNPHARMA.NS, DIVISLAB.NS

In [39]:
tickers = ['INFY.NS', 'TCS.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'ITC.NS', 'HINDUNILVR.NS', 'RELIANCE.NS', 'ONGC.NS', 'SUNPHARMA.NS', 'DIVISLAB.NS']
tickers

['INFY.NS',
 'TCS.NS',
 'HDFCBANK.NS',
 'ICICIBANK.NS',
 'ITC.NS',
 'HINDUNILVR.NS',
 'RELIANCE.NS',
 'ONGC.NS',
 'SUNPHARMA.NS',
 'DIVISLAB.NS']

In [40]:
# Date range
start = '2023-10-01'
end = '2025-11-12'

# assets.sort()

# # Downloading data
# data = yf.download(tickers, start = start, end = end)

# # data = data.loc[:,('Close', slice(None))]
# data = data['Close'].round(2)

# assets_1 = ['INFY', 'TCS', 'HDFCBANK', 'ICICIBANK', 'ITC', 'HINDUNILVR', 'RELIANCE', 'ONGC', 'SUNPHARMA', 'DIVISLAB']

# data.columns = assets_1

In [41]:
# Download Close prices
raw = yf.download(tickers, start=start, end=end, progress=False)
prices = raw['Close'].copy()

# Optional: rename columns to shorter symbols (strip .NS) if you prefer
short_names = [t.split('.')[0] for t in tickers]
prices.columns = short_names

# Reindex to business days and forward fill missing days (holidays/weekends)
bdays = pd.bdate_range(start=start, end=end)
prices = prices.reindex(bdays).ffill()

# Keep last 504 business days (~2 years = 1.5y train + 0.5y test)
prices = prices.tail(504)

# Save prices
prices.to_csv(os.path.join(OUTDIR, "prices.csv"))

print("Prices shape:", prices.shape)
prices.head()

Prices shape: (504, 10)


,INFY,TCS,HDFCBANK,ICICIBANK,ITC,HINDUNILVR,RELIANCE,ONGC,SUNPHARMA,DIVISLAB
2023-12-08,3642.704102,804.588867,2424.476562,994.548035,1403.895142,420.429565,178.073898,1218.853638,1213.578125,3439.028564
2023-12-11,3631.573242,803.518188,2406.549805,1000.697266,1401.400146,423.518585,179.755142,1220.640503,1219.077393,3454.390137
2023-12-12,3608.916260,795.536499,2405.685059,999.319885,1389.678833,424.127014,177.574066,1203.120117,1196.098267,3482.079590
2023-12-13,3617.227051,793.735779,2414.480225,997.548889,1364.211670,426.513977,175.529358,1208.033691,1210.926636,3407.594238
2023-12-14,3644.138672,803.104492,2418.757568,1016.980408,1413.592407,430.679413,178.073898,1223.022827,1209.159058,3477.480225


# Train/Test split: 1.5 years (378 bd) train, 6 months (126 bd) test

In [ ]:
# lengths
train_days = 378   # ~1.5 years
test_days  = 126   # ~6 months
assert train_days + test_days == 504

train_prices = prices.iloc[:-test_days].copy()
test_prices  = prices.iloc[-test_days:].copy()

print("Train:", train_prices.index.min(), "to", train_prices.index.max(), train_prices.shape)
print("Test :", test_prices.index.min(),  "to", test_prices.index.max(),  test_prices.shape)

train_prices.head()
